In [1]:
from dask.distributed import Client
client = Client()

In [ ]:
sfile = "trip_data/trip_data_*.csv"

In [ ]:
# Modify row trip_time_in_secs and convert to minutes if greater than 60, store in new column
def transform(row):
    return row.trip_time_in_secs/60

In [ ]:
import dask.dataframe as dd
import pandas as pd
import numpy as np
ddf = dd.read_csv(sfile)

In [ ]:
ddf['mins'] = ddf.apply(transform, axis=1, meta=('mins', 'f8'))

In [ ]:
ddf.compute()

In [ ]:
ddf.dtypes

## map_partitions with read_csv only

In [ ]:
import dask.dataframe as dd
import pandas as pd

In [ ]:
sfile = "trip_data/trip_data_1*.csv"
ddf2 = dd.read_csv(sfile)

In [ ]:
def convert_to_mins(secs):
    return secs / 60

# Take the trip_time_in_secs column, convert to minutes, store in a new column
def transform(df):
    df["mins"] = df.trip_time_in_secs.map(convert_to_mins)
    return df

In [ ]:
dtypes = {
    'medallion' : '', 
    'hack_license': '',
    'vendor_id': '',
    'rate_code': '',
    'store_and_fwd_flag': int(),
    'pickup_datetime': float(),
    'dropoff_datetime': float(),
    'passenger_count': int(),
    'trip_time_in_secs': float(),
    'trip_distance': float(),
    'pickup_longitude': float(),
    'pickup_latitude': float(),
    'dropoff_longitude': float(),
    'dropoff_latitude': float(),
    'mins': float()
}
                        


In [ ]:
new_ddf = ddf2.map_partitions(transform, meta=(pd.DataFrame(dtypes, index=[])))

In [ ]:
%%time
new_ddf.compute()

In [ ]:
size = new_ddf.size
size.visualize()

In [ ]:
size.compute()

## map_partitions with read_parquet

In [2]:
import dask.dataframe as dd
import pandas as pd

In [3]:
sfile = "trip_data/trip_data_1*.csv"
ddf2 = dd.read_csv(sfile)

In [4]:
dest = "trip_data_parquet/"
ddf2.to_parquet(path=dest, engine='fastparquet', compute=True)

In [5]:
parquet = dd.read_parquet(path=dest, engine='fastparquet')

In [6]:
def convert_to_mins(secs):
    return secs / 60

# Take the trip_time_in_secs column, convert to minutes, store in a new column
def transform(df):
    df["mins"] = df.trip_time_in_secs.map(convert_to_mins)
    return df

In [7]:
dtypes = {
    'medallion' : '', 
    'hack_license': '',
    'vendor_id': '',
    'rate_code': '',
    'store_and_fwd_flag': int(),
    'pickup_datetime': float(),
    'dropoff_datetime': float(),
    'passenger_count': int(),
    'trip_time_in_secs': float(),
    'trip_distance': float(),
    'pickup_longitude': float(),
    'pickup_latitude': float(),
    'dropoff_longitude': float(),
    'dropoff_latitude': float(),
    'mins': float()
}
      

In [8]:
new_ddf = ddf2.map_partitions(transform, meta=(pd.DataFrame(dtypes, index=[])))

In [9]:
%%time
new_ddf.compute()

CPU times: user 52.7 s, sys: 55.5 s, total: 1min 48s
Wall time: 3min 37s


,medallion,hack_license,vendor_id,rate_code,store_and_fwd_flag,pickup_datetime,dropoff_datetime,passenger_count,trip_time_in_secs,trip_distance,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,mins
0,89D227B655E5C82AECF13C3F540D4CF4,BA96DE419E711691B9445D6A6307C170,CMT,1,N,2013-01-01 15:11:48,2013-01-01 15:18:10,4,382,1.00,-73.978165,40.757977,-73.989838,40.751171,6.366667
1,0BD7C8F5BA12B88E0B67BED28BEA73D8,9FD8F69F0804BDB5549F40E9DA1BE472,CMT,1,N,2013-01-06 00:18:35,2013-01-06 00:22:54,1,259,1.50,-74.006683,40.731781,-73.994499,40.750660,4.316667
2,0BD7C8F5BA12B88E0B67BED28BEA73D8,9FD8F69F0804BDB5549F40E9DA1BE472,CMT,1,N,2013-01-05 18:49:41,2013-01-05 18:54:23,1,282,1.10,-74.004707,40.737770,-74.009834,40.726002,4.700000
3,DFD2202EE08F7A8DC9A57B02ACB81FE2,51EE87E3205C985EF8431D850C786310,CMT,1,N,2013-01-07 23:54:15,2013-01-07 23:58:20,2,244,0.70,-73.974602,40.759945,-73.984734,40.759388,4.066667
4,DFD2202EE08F7A8DC9A57B02ACB81FE2,51EE87E3205C985EF8431D850C786310,CMT,1,N,2013-01-07 23:25:03,2013-01-07 23:34:24,1,560,2.10,-73.976250,40.748528,-74.002586,40.747868,9.333333
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
140167,803FA45BD7CC003F460410D5D9EE4D3A,6C1B453B66ACE118D95802CA27053C2B,VTS,1,NaN,2013-12-03 17:12:00,2013-12-03 17:42:00,1,1800,10.01,-73.873077,40.774055,-73.973000,40.756882,30.000000
140168,DBE7422E79192B5F654FE1FE02526125,3158E46011E95D376541A461895A7730,VTS,1,NaN,2013-12-06 06:55:00,2013-12-06 07:04:00,1,540,2.17,-73.996338,40.733047,-73.978561,40.750282,9.000000
140169,ABC075C8871353E9F12ADEB5F1E27C2A,2E24AB48335EDC44EADE1CE29F6BCBF4,VTS,1,NaN,2013-12-06 06:57:00,2013-12-06 07:06:00,3,540,1.69,-73.972183,40.794407,-73.972336,40.796398,9.000000
140170,1FB2BF12B504498BC8B7B860294CD372,03D43975C87C8B5CFBDA1FF33B3AD743,VTS,1,NaN,2013-12-06 07:04:00,2013-12-06 07:06:00,1,120,0.70,0.000000,0.000000,0.000000,0.000000,2.000000
